In [41]:
import pandas as pd
import numpy as np

In [42]:
df = pd.DataFrame({"A": [1,2,3,4], "B": [1,2,np.nan,4]})
df2 = pd.DataFrame({"A": [1,2,3,4], "B": [1,2,3,4]})

In [43]:
def non_or_empty(x):
    def bitwise_non_or_empty(x):
        boo = False
        if np.isnan(x): 
            boo = True
        if x == None:
            boo = True
        if x == '':
            boo = True
        return boo
    boo = x.apply(
        lambda x: bitwise_non_or_empty(x))
    return boo

In [44]:
def join_and_update(left, right, left_on, right_on,
                    left_update, right_update, key=""):
    df_join = left.merge(right, how="left",
                         left_on=left_on, right_on=right_on,
                         suffixes=['','___y'])
    check = (non_or_empty(df_join[left_update])) & (~non_or_empty(df_join[right_update+'___y']))
    df_join.loc[check, 'Update_on'] = key
    df_join.loc[check, left_update] = df_join[right_update + '___y']
    df_join = df_join[[i for i in df_join.columns if not i.endswith('___y')]]
    return df_join

In [45]:
join_and_update(df, df2, left_on="A", right_on="A", left_update="B", right_update="B", key="Joined")

,A,B,Update_on
0,1,1.0,NaN
1,2,2.0,NaN
2,3,3.0,Joined
3,4,4.0,NaN
